In [110]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error


In [111]:
retail_data = pd.read_csv("valid_retail_data.csv")

In [112]:
retail_data.head()

,Invoice_ID,Customer_ID,Branch,City,Customer_Type,Product_Category,Product,Product_General_Category,Product_SubCategory,Product_Brand,...,Weight_lb,Weight_oz,Weight_g,Weight_fl_oz,Weight_ml,Weight_L,Weight_gallon,Weight_all,Expiry_Date,Is_Expired
0,US265149,CUST573806,Northeast,San Diego,Regular,Beverages,Cold Pressed Juice,Consumer Goods,Cold Beverages,Organic Valley,...,0.035,0.54,16.0,0.54,16.0,0.016,0.004,16.0g/16.0ml,2022-01-03,1
1,US756849,CUST101861,Southeast,Phoenix,Regular,Beverages,Sparkling Water 12-Pack,Consumer Goods,Hot Beverages,Nature's Best,...,0.317,4.87,144.0,4.87,144.0,0.144,0.038,144.0g/144.0ml,2021-08-06,1
2,US148893,CUST144006,West,New York,Regular,Beverages,Premium Coffee,Consumer Goods,Cold Beverages,Pure Life,...,0.026,0.41,12.0,0.41,12.0,0.012,0.003,12.0g/12.0ml,2022-03-01,1
3,US958245,CUST623978,Central,San Diego,Regular,Beverages,Organic Tea,Consumer Goods,Hot Beverages,Pure Life,...,0.044,0.71,20.0,0.00,20.0,0.020,0.005,20.0g/20.0ml,2021-07-22,1
4,US283003,CUST691904,West,Los Angeles,Member,Beverages,Cold Pressed Juice,Consumer Goods,Cold Beverages,Wholesome Co.,...,0.035,0.54,16.0,0.54,16.0,0.016,0.004,16.0g/16.0ml,2022-01-18,1


In [113]:
def handle_missing_values(daily):
    """
    Handle missing values with appropriate strategies for different types of data
    """
    # Create a copy to avoid SettingWithCopyWarning
    daily = daily.copy()

    # Group columns by filling strategy
    price_columns = ['Current_Unit_Price', 'Unit_Cost']
    inventory_columns = ['Current_Stock', 'Beginning Inventory Stock', 'Ending Inventory Stock']
    sales_columns = ['Quantity', 'Revenue']

    # For prices: use last known price
    for col in price_columns:
        if col in daily.columns:
            temp = daily.groupby('Product')[col].ffill()
            daily[col] = temp
            temp = daily.groupby('Product')[col].bfill()
            daily[col] = temp

    # For inventory: interpolate between known points
    for col in inventory_columns:
        if col in daily.columns:
            # First interpolate
            daily[col] = daily[col].interpolate(method='linear')
            # Then handle any remaining missing values
            temp = daily.groupby('Product')[col].ffill()
            daily[col] = temp
            temp = daily.groupby('Product')[col].bfill()
            daily[col] = temp

    # For sales: use 0 for missing values
    for col in sales_columns:
        if col in daily.columns:
            daily[col] = daily[col].fillna(0)

    # For remaining numeric columns
    numeric_cols = daily.select_dtypes(include=['float64', 'int64']).columns
    remaining_cols = [col for col in numeric_cols
                      if col not in price_columns + inventory_columns + sales_columns]

    # Handle remaining columns
    for col in remaining_cols:
        temp = daily.groupby('Product')[col].ffill()
        daily[col] = temp
        temp = daily.groupby('Product')[col].bfill()
        daily[col] = temp

    return daily

In [114]:
def process_data(df: pd.DataFrame) -> pd.DataFrame:
    """Process and enhance retail data with inventory and sales features"""

    # Convert date and ensure proper format
    df['Date'] = pd.to_datetime(df['Date'])

    # Calculate key metrics for each product
    daily = df.groupby(['Date', 'Product_Category', 'Product_SubCategory', 'Product', 'Product_Brand']).agg({
        # Sum for volumetric/monetary values
        'Quantity': 'sum',  # Total units sold
        'Cost_of_Goods': 'sum',  # Total cost of goods
        'Profit': 'sum',  # Total profit

        # Last value for inventory positions
        'Beginning Inventory Stock': 'last',  # Opening inventory
        'Ending Inventory Stock': 'last',  # Closing inventory
        'Current_Stock': 'last',  # Current stock level
        'Reorder_Point': 'last',  # Reorder point

        # Last for calculated ratios (since these are already computed)
        'Weekly_itr': 'last',  # Weekly inventory turnover
        'Monthly_itr': 'last',  # Monthly inventory turnover
        'Annual_itr': 'last',  # Annual inventory turnover
        'Average_inventory_period': 'last',  # Days to sell inventory

        # Mean for pricing data
        'Current_Unit_Price': 'mean',  # Average selling price
        'Unit_Cost': 'mean',  # Average unit cost

        # Mean for environmental/external factors
        'Temperature': 'mean',
        'Humidity': 'mean',
        'Precipitation': 'mean',
        'Consumer_Confidence_Index': 'mean',
        'Traffic_Level': 'mean',

        # Last for capacity metrics
        'Store_Capacity_Utilization': 'last',

        # Mean for competitive metrics
        'Online_Competition_Index': 'mean',

        # Last for seasonal factors (already calculated)
        'Seasonal_Factor': 'last',

        # Mean for customer feedback
        'Rating': 'mean'
    }).reset_index()

    # Add time-based features
    daily['Year'] = daily['Date'].dt.year
    daily['Month'] = daily['Date'].dt.month
    daily['Day'] = daily['Date'].dt.day

    # Calculate inventory metrics
    daily['Inventory_Change'] = daily['Ending Inventory Stock'] - daily['Beginning Inventory Stock']
    daily['Stock_Coverage_Days'] = daily['Current_Stock'] * daily['Average_inventory_period'] / daily['Quantity'].where(
        daily['Quantity'] > 0, 1)

    # We take the average value price by category and then divide 'Current_Unit_Price'/ 'Market_Avg_Price' and get competitive prices by date and category
    daily['Weighted_Avg_Price'] = (
            (daily['Quantity'] * daily['Current_Unit_Price']).groupby(
                [daily['Date'],
                 daily['Product_Category'],
                 daily['Product_SubCategory'],
                 daily['Product'],
                 daily['Product_Brand']])
            .transform('sum')
            / daily['Quantity'].groupby(
        [daily['Date'],
         daily['Product_Category'],
         daily['Product_SubCategory'],
         daily['Product'],
         daily['Product_Brand']]).transform('sum')
    )  

    # Revenue
    daily['Revenue'] = daily['Quantity'] * daily['Weighted_Avg_Price']

    daily = handle_missing_values(daily)
    daily = daily.sort_values(['Product', 'Date'])
    
    # These rolling statistics features are crucial for understanding product performance over different time windows.
    for window in [7, 30]:
        # Rolling sales metrics
        daily[f'Rolling_{window}d_Quantity'] = daily.groupby('Product_Category')['Quantity'].transform(
            lambda x: x.rolling(window, min_periods=1).mean())

        daily[f'Rolling_{window}d_Revenue'] = daily.groupby('Product')['Revenue'].transform(
            lambda x: x.rolling(window, min_periods=1).mean())

        # Rolling inventory metrics
        daily[f'Rolling_{window}d_ITR'] = daily.groupby('Product')['Weekly_itr'].transform(
            lambda x: x.rolling(window, min_periods=1).mean())

        daily[f'Rolling_{window}d_Stock'] = daily.groupby('Product')['Current_Stock'].transform(
            lambda x: x.rolling(window, min_periods=1).mean())

    # Measure demand stability
    # Identify unpredictable products
    daily['Demand_Volatility'] = daily.groupby('Product')['Quantity'].transform(
        lambda x: x.rolling(30, min_periods=1).std())

    # Compute z-scores for Temperature and Seasonal_Factor before multiplying
    daily['Temp_Z'] = (daily['Temperature'] - daily['Temperature'].mean()) / daily['Temperature'].std()
    daily['Seasonal_Factor_Z'] = (daily['Seasonal_Factor'] - daily['Seasonal_Factor'].mean()) / daily[
        'Seasonal_Factor'].std()

    daily['Seasonal_Strength'] = daily['Seasonal_Factor_Z'] * daily['Temp_Z']

    # Lagged features with NaN handling
    for product_group in daily.groupby('Product'):
        product_data = product_group[1].sort_values('Date')
        
        # Handle Quantity lags
        daily.loc[product_data.index, 'Quantity_lag_7'] = product_data['Quantity'].shift(7)
        daily.loc[product_data.index, 'Quantity_lag_30'] = product_data['Quantity'].shift(30)
        
        # Handle Revenue lags
        daily.loc[product_data.index, 'Revenue_lag_7'] = product_data['Revenue'].shift(7)
        daily.loc[product_data.index, 'Revenue_lag_30'] = product_data['Revenue'].shift(30)

    # Fill NaN values in lagged features with forward fill then backward fill
    lag_columns = ['Quantity_lag_7', 'Quantity_lag_30', 'Revenue_lag_7', 'Revenue_lag_30']
    for col in lag_columns:
        daily[col] = daily.groupby('Product')[col].transform(
            lambda x: x.fillna(method='ffill').fillna(method='bfill')
        )

    # EWMA calculations with NaN handling
    ewma_pairs = [
        ('Quantity', 'Quantity_EWMA'),
        ('Revenue', 'Revenue_EWMA'),
        ('Current_Unit_Price', 'Price_EWMA')
    ]

    for source_col, target_prefix in ewma_pairs:
        for window in [7, 30]:
            col_name = f'{target_prefix}_{window}d'
            daily[col_name] = daily.groupby('Product')[source_col].transform(
                lambda x: x.ewm(span=window, adjust=False, min_periods=1).mean()
            )
            # Fill remaining NaNs with forward fill then backward fill
            daily[col_name] = daily.groupby('Product')[col_name].transform(
                lambda x: x.fillna(method='ffill').fillna(method='bfill')
            )

    # Final check for any remaining NaNs in critical columns
    critical_columns = (
        lag_columns + 
        [col for col in daily.columns if 'EWMA' in col] +
        ['Quantity', 'Revenue']
    )
    
    for col in critical_columns:
        if daily[col].isna().any():
            # If any NaNs remain, fill with 0
            daily[col] = daily[col].fillna(0)
            print(f"Warning: Found and filled NaN values in {col} with 0")

    return daily

In [115]:
data_after_processing = process_data(retail_data)

/var/folders/xp/cmbd5jd15_v_gbf4400k4jgh0000gn/T/ipykernel_23160/1820972946.py:127: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/xp/cmbd5jd15_v_gbf4400k4jgh0000gn/T/ipykernel_23160/1820972946.py:127: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/xp/cmbd5jd15_v_gbf4400k4jgh0000gn/T/ipykernel_23160/1820972946.py:127: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/xp/cmbd5jd15_v_gbf4400k4jgh0000gn/T/ipykernel_23160/1820972946.py:127: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/xp/cmbd5jd15_v_gbf4400k4jgh0000gn/T/ipykernel_23160/1820972946.py:145: FutureWarning:

Series.fillna with 'meth

In [116]:
data_after_processing

,Date,Product_Category,Product_SubCategory,Product,Product_Brand,Quantity,Cost_of_Goods,Profit,Beginning Inventory Stock,Ending Inventory Stock,...,Quantity_lag_7,Quantity_lag_30,Revenue_lag_7,Revenue_lag_30,Quantity_EWMA_7d,Quantity_EWMA_30d,Revenue_EWMA_7d,Revenue_EWMA_30d,Price_EWMA_7d,Price_EWMA_30d
48,2021-01-01,Home & Kitchen,Eco-Friendly Products,Bamboo Utensil Set,Green Living,7,200.20,4.13,249,242,...,7.0,7.0,204.33,204.33,7.000000,7.000000,204.330000,204.330000,29.190000,29.190000
49,2021-01-01,Home & Kitchen,Eco-Friendly Products,Bamboo Utensil Set,Home Essentials,6,133.68,17.46,555,549,...,7.0,7.0,204.33,204.33,6.750000,6.935484,191.032500,200.898387,28.190000,28.931935
50,2021-01-01,Home & Kitchen,Eco-Friendly Products,Bamboo Utensil Set,Kitchen Pro,10,100.52,10.60,313,309,...,7.0,7.0,204.33,204.33,7.562500,7.133195,173.886875,195.837201,24.203750,27.855359
53,2021-01-01,Home & Kitchen,Kitchen Tools,Bamboo Utensil Set,EcoHome,14,188.80,21.54,530,525,...,7.0,7.0,204.33,204.33,9.171875,7.576214,185.120156,197.319962,22.060313,27.066626
54,2021-01-01,Home & Kitchen,Kitchen Tools,Bamboo Utensil Set,Home Essentials,7,177.59,10.71,401,394,...,7.0,7.0,204.33,204.33,8.628906,7.539039,185.915117,196.738029,23.270234,27.055876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2882,2023-12-31,Grocery,Organic Foods,Trail Mix,Organic Feast,10,221.84,4.94,328,326,...,3.0,5.0,21.84,76.65,8.839875,6.449068,181.894329,128.906747,20.996325,20.095609
2883,2023-12-31,Grocery,Organic Foods,Trail Mix,Pure Harvest,4,32.28,1.48,566,562,...,9.0,2.0,120.33,34.76,7.629906,6.291063,144.860747,122.768248,17.857244,19.343634
2887,2023-12-31,Grocery,Snacks,Trail Mix,Health Valley,1,5.09,0.65,252,251,...,4.0,3.0,113.40,88.23,5.972430,5.949704,110.080560,115.218038,14.827933,18.465980
2888,2023-12-31,Grocery,Snacks,Trail Mix,Nature's Choice,5,125.05,16.90,360,355,...,3.0,5.0,82.20,47.80,5.729322,5.888433,118.047920,116.942681,18.218450,19.106240


In [117]:
def create_key_metrics_visualizations(daily, product_category=None):
    """
    Create comprehensive visualizations of key business metrics
    """
    # Filter data if category specified
    plot_data = daily.copy()
    if product_category:
        plot_data = daily[daily['Product_Category'] == product_category].copy()

    # Create subplots
    fig = make_subplots(
        rows=4, cols=2,
        subplot_titles=(
            'Inventory Turnover Metrics', 'Stock Levels',
            'Price and Revenue Analysis', 'Seasonal Impact',
            'Customer Response', 'Demand Patterns',
            'Environmental Factors', 'Market Position'
        ),
        vertical_spacing=0.12
    )

    # 1. Inventory Turnover Metrics (row=1, col=1)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Weekly_itr'],
            name='Weekly ITR',
            line=dict(color='blue')
        ),
        row=1, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Monthly_itr'],
            name='Monthly ITR',
            line=dict(color='red')
        ),
        row=1, col=1
    )

    # 2. Stock Levels (row=1, col=2)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Current_Stock'],
            name='Current Stock',
            line=dict(color='green')
        ),
        row=1, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Reorder_Point'],
            name='Reorder Point',
            line=dict(color='red', dash='dash')
        ),
        row=1, col=2
    )

    # 3. Price and Revenue (row=2, col=1)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Current_Unit_Price'],
            name='Unit Price',
            line=dict(color='purple')
        ),
        row=2, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Revenue'],
            name='Revenue',
            line=dict(color='orange'),
            yaxis='y3'
        ),
        row=2, col=1
    )

    # 4. Seasonal Impact (row=2, col=2)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Seasonal_Factor'],
            name='Seasonal Factor',
            line=dict(color='brown')
        ),
        row=2, col=2
    )

    # 5. Customer Response (row=3, col=1)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Rating'],
            name='Customer Rating',
            line=dict(color='gold')
        ),
        row=3, col=1
    )

    # 6. Demand Patterns (row=3, col=2)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Quantity'],
            name='Daily Sales',
            line=dict(color='skyblue')
        ),
        row=3, col=2
    )
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Quantity_EWMA_30d'],
            name='30-Day Trend',
            line=dict(color='navy', dash='dash')
        ),
        row=3, col=2
    )

    # 7. Environmental Factors (row=4, col=1)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Temperature'],
            name='Temperature',
            line=dict(color='red')
        ),
        row=4, col=1
    )
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Humidity'],
            name='Humidity',
            line=dict(color='blue')
        ),
        row=4, col=1
    )

    # 8. Market Position (row=4, col=2)
    fig.add_trace(
        go.Scatter(
            x=plot_data['Date'],
            y=plot_data['Online_Competition_Index'],
            name='Competition Index',
            line=dict(color='purple')
        ),
        row=4, col=2
    )

    # Update layout
    fig.update_layout(
        height=1600,
        width=1200,
        title_text=f"Key Business Metrics Analysis {'for ' + product_category if product_category else ''}",
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Update y-axes titles
    fig.update_yaxes(title_text="ITR Value", row=1, col=1)
    fig.update_yaxes(title_text="Units", row=1, col=2)
    fig.update_yaxes(title_text="Price ($)", row=2, col=1)
    fig.update_yaxes(title_text="Factor", row=2, col=2)
    fig.update_yaxes(title_text="Rating", row=3, col=1)
    fig.update_yaxes(title_text="Units", row=3, col=2)
    fig.update_yaxes(title_text="Value", row=4, col=1)
    fig.update_yaxes(title_text="Index", row=4, col=2)

    return fig

In [118]:
def create_category_comparison(daily):
    """
    Create visualizations comparing different product categories
    """
    # Aggregate metrics by category
    category_metrics = daily.groupby('Product_Category').agg({
        'Quantity': 'sum',
        'Revenue': 'sum',
        'Weekly_itr': 'mean',
        'Rating': 'mean',
        'Demand_Volatility': 'mean'
    }).reset_index()

    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Total Sales by Category',
            'Average ITR by Category',
            'Customer Rating by Category',
            'Demand Volatility by Category'
        )
    )

    # 1. Sales by Category
    fig.add_trace(
        go.Bar(
            x=category_metrics['Product_Category'],
            y=category_metrics['Quantity'],
            name='Total Sales'
        ),
        row=1, col=1
    )

    # 2. ITR by Category
    fig.add_trace(
        go.Bar(
            x=category_metrics['Product_Category'],
            y=category_metrics['Weekly_itr'],
            name='Avg ITR'
        ),
        row=1, col=2
    )

    # 3. Rating by Category
    fig.add_trace(
        go.Bar(
            x=category_metrics['Product_Category'],
            y=category_metrics['Rating'],
            name='Avg Rating'
        ),
        row=2, col=1
    )

    # 4. Volatility by Category
    fig.add_trace(
        go.Bar(
            x=category_metrics['Product_Category'],
            y=category_metrics['Demand_Volatility'],
            name='Demand Volatility'
        ),
        row=2, col=2
    )

    # Update layout
    fig.update_layout(
        height=1000,
        width=1200,
        title_text="Category Performance Comparison",
        showlegend=False
    )

    # Update axes
    for i in range(1, 3):
        for j in range(1, 3):
            fig.update_xaxes(tickangle=45, row=i, col=j)

    return fig

In [119]:
def prepare_data(train_data, test_data, feature_cols, target_col):
    """
    Prepare and normalize data for model training
    """
    # Create scalers
    feature_scaler = StandardScaler()
    target_scaler = StandardScaler()

    # Create copies to avoid modifying original data
    train_scaled = train_data.copy()
    test_scaled = test_data.copy()

    # Get numeric features (excluding Date)
    numeric_features = [col for col in feature_cols if col != 'Date']

    # Scale features
    train_scaled[numeric_features] = feature_scaler.fit_transform(train_data[numeric_features])
    test_scaled[numeric_features] = feature_scaler.transform(test_data[numeric_features])

    # Scale target
    train_scaled[target_col] = target_scaler.fit_transform(train_data[[target_col]])
    test_scaled[target_col] = target_scaler.transform(test_data[[target_col]])

    return train_scaled, test_scaled, feature_scaler, target_scaler

In [120]:
def analyze_normalization_impact(original_data, normalized_data, feature_cols):
    """
    Analyze the impact of normalization on features
    """
    impact = {}

    # Create figure
    fig = go.Figure()

    for col in feature_cols:
        if col != 'Date':
            # Calculate statistics
            orig_std = original_data[col].std()
            norm_std = normalized_data[col].std()

            impact[col] = {
                'original_std': orig_std,
                'normalized_std': norm_std,
                'scale_factor': orig_std / norm_std if norm_std != 0 else 0
            }

            # Add box plots for original and normalized data
            fig.add_trace(go.Box(
                y=original_data[col],
                name=f"{col} (Original)",
                boxpoints='outliers'
            ))

            fig.add_trace(go.Box(
                y=normalized_data[col],
                name=f"{col} (Normalized)",
                boxpoints='outliers'
            ))

    # Update layout
    fig.update_layout(
        title='Feature Distribution Before/After Normalization',
        yaxis_title='Value',
        showlegend=True,
        height=800,
        width=1200,
        boxmode='group'
    )

    return impact, fig

In [121]:
def train_models_with_cv(train_data, test_data, feature_cols, target_cols=None, n_splits=5):
    """
    Train and cross-validate Prophet, SARIMAX, and LightGBM models with normalized data
    """
    results = {}

    for target_col in target_cols:
        print(f"\nTraining models for {target_col}")

        # normalize data
        train_scaled, test_scaled, feature_scaler, target_scaler = prepare_data(
            train_data, test_data, feature_cols, target_col
        )
        # analayze normalize data
        analyze_normalization_impact(train_data, train_scaled, feature_cols)
        analyze_normalization_impact(test_data, test_scaled, feature_cols)

        # Get numeric features (excluding Date)
        numeric_features = [col for col in feature_cols if col != 'Date']

        def calculate_mape(y_true, y_pred):
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            mask = y_true != 0
            return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

        # 1. LightGBM with normalized data
        def train_lightgbm(train_df, test_df):
            params = {
                'objective': 'regression',
                'metric': 'mape',
                'boosting_type': 'gbdt',
                'num_leaves': 31,
                'learning_rate': 0.05,
                'feature_fraction': 0.9
            }

            # Use normalized features and target
            train_set = lgb.Dataset(
                train_df[numeric_features],
                train_df[target_col]
            )
            model = lgb.train(params, train_set, num_boost_round=100)

            # Predict on normalized data
            forecast = model.predict(test_df[numeric_features])
            return model, forecast

        # 2. Prophet with normalized data
        def train_prophet(train_df, test_df):
            # Use normalized target
            prophet_train = pd.DataFrame({
                'ds': train_df['Date'],
                'y': train_df[target_col].values
            })

            model = Prophet(
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=True,
                seasonality_mode='multiplicative',
                changepoint_prior_scale=0.05
            )

            # Add normalized features as regressors
            for feature in numeric_features:
                model.add_regressor(feature)
                prophet_train[feature] = train_df[feature].values

            model.fit(prophet_train)

            # Prepare test data with normalized features
            prophet_test = pd.DataFrame({'ds': pd.to_datetime(test_df['Date'])})
            for feature in numeric_features:
                prophet_test[feature] = test_df[feature].values

            forecast = model.predict(prophet_test)['yhat'].values
            return model, forecast

        # 3. SARIMAX with normalized data
        def train_sarimax(train_df, test_df):
            train_values = train_df[target_col].values.reshape(-1, 1)
            # Use normalized target
            model = SARIMAX(
                train_values,
                order=(1, 1, 1),
                seasonal_order=(1, 1, 1, 12),
                enforce_stationarity=False,
                enforce_invertibility=False
            )

            fitted_model = model.fit(disp=False)
            forecast = fitted_model.forecast(steps=len(test_df))
            return fitted_model, forecast

        # Initialize CV scores
        cv_scores = {
            'lightgbm': {'mae': [], 'mape': []},
            'prophet': {'mae': [], 'mape': []},
            'sarimax': {'mae': [], 'mape': []}
        }

        # Cross Validation on normalized data
        tscv = TimeSeriesSplit(n_splits=n_splits)

        for train_idx, val_idx in tscv.split(train_scaled):
            cv_train = train_scaled.iloc[train_idx]
            cv_val = train_scaled.iloc[val_idx]

            for model_name, train_func in {
                'lightgbm': train_lightgbm,
                'prophet': train_prophet,
                'sarimax': train_sarimax
            }.items():
                try:
                    model, forecast = train_func(cv_train, cv_val)

                    # Transform predictions back to original scale
                    forecast = target_scaler.inverse_transform(
                        forecast.reshape(-1, 1)
                    ).flatten()

                    true_values = target_scaler.inverse_transform(
                        cv_val[target_col].values.reshape(-1, 1)
                    ).flatten()

                    # Calculate metrics on original scale
                    cv_scores[model_name]['mae'].append(
                        mean_absolute_error(true_values, forecast)
                    )
                    cv_scores[model_name]['mape'].append(
                        calculate_mape(true_values, forecast)
                    )

                except Exception as e:
                    print(f"Error in {model_name} CV: {str(e)}")

        # Train final models on normalized data
        final_models = {}
        final_forecasts = {}

        for model_name, train_func in {
            'lightgbm': train_lightgbm,
            'prophet': train_prophet,
            'sarimax': train_sarimax
        }.items():
            try:
                model, forecast = train_func(train_scaled, test_scaled)

                # Transform predictions back to original scale
                forecast = target_scaler.inverse_transform(
                    forecast.reshape(-1, 1)
                ).flatten()

                final_models[model_name] = model
                final_forecasts[model_name] = forecast

            except Exception as e:
                print(f"Error in final {model_name} training: {str(e)}")

        results[target_col] = {
            'models': final_models,
            'forecasts': final_forecasts,
            'cv_scores': cv_scores,
            'scalers': {
                'feature_scaler': feature_scaler,
                'target_scaler': target_scaler
            }
        }

    return results

In [122]:
def prepare_forecasting_data(daily_data, test_size=0.2):
    """
    Prepare data for forecasting by splitting into train and test sets
    
    Parameters:
    -----------
    daily_data: pd.DataFrame
        Processed daily data from process_data function
    test_size: float
        Proportion of data to use for testing (default: 0.2 = 20%)
    """
    # Sort by date to ensure temporal ordering
    daily_data = daily_data.sort_values('Date')

    # Define feature columns
    feature_cols = [
        # Time features
        'Year', 'Month', 'Day', 'Is_Weekend', 'Is_Holiday',
        'Days_To_Next_Holiday', 'Days_To_Payday',

        # Product performance metrics
        'current_Margin', 'Profit', 'Rating',
        
        # Inventory metrics
        'Weekly_itr', 'Monthly_itr', 'Annual_itr',
        'Current_Stock', 'Reorder_Point',
        'Stock_Coverage_Days', 'Inventory_Change',
        'Beginning Inventory Stock', 'Ending Inventory Stock',
        'Average_inventory_period', 'Turnover_Ratio_Change_Flag',

        # Rolling metrics
        'Rolling_7d_Quantity', 'Rolling_30d_Quantity',
        'Rolling_7d_Revenue', 'Rolling_30d_Revenue',
        'Rolling_7d_ITR', 'Rolling_30d_ITR',
        'Rolling_7d_Stock', 'Rolling_30d_Stock',

        # Price metrics
        'Current_Unit_Price', 'Unit_Cost', 
        'Weighted_Avg_Price', 'Proposed_Unit_Price',
        'Price_EWMA_7d', 'Price_EWMA_30d',

        # Lagged features
        'Quantity_lag_7', 'Quantity_lag_30',
        'Revenue_lag_7', 'Revenue_lag_30',

        # EWMA features
        'Quantity_EWMA_7d', 'Quantity_EWMA_30d',
        'Revenue_EWMA_7d', 'Revenue_EWMA_30d',

        # Environmental factors
        'Temperature', 'Humidity', 'Precipitation',
        'Temp_Z', 'Seasonal_Factor_Z', 'Seasonal_Strength',
        'Seasonal_Factor', 'Seasonal_Factor_Change_Flag',

        # Market indicators
        'Consumer_Confidence_Index', 'product_cci',
        'Traffic_Level', 'Traffic_Level_Change_Flag',
        'Store_Capacity_Utilization', 
        'Online_Competition_Index',
        'Competitor_Promotion', 'Marketing_Campaign',
        'Local_Events'
    ]
    
    available_features = []
    for col in feature_cols:
        if col in daily_data.columns:
            # Check if column has non-null values
            if daily_data[col].notna().any():
                available_features.append(col)
            else:
                print(f"Warning: Column {col} exists but contains only null values - excluding")
        else:
            print(f"Warning: Column {col} not found in dataset - excluding")
    
    feature_cols = available_features
    print(f"\nUsing {len(feature_cols)} features for modeling")
    
    # Calculate split point
    split_date = daily_data['Date'].max() - pd.Timedelta(days=int(len(daily_data.Date.unique()) * test_size))

    # Split data
    train_data = daily_data[daily_data['Date'] <= split_date].copy()
    test_data = daily_data[daily_data['Date'] > split_date].copy()

    print(f"Train period: {train_data['Date'].min()} to {train_data['Date'].max()}")
    print(f"Test period: {test_data['Date'].min()} to {test_data['Date'].max()}")
    print(f"Number of features: {len(feature_cols)}")

    return train_data, test_data, feature_cols

In [123]:
def run_forecasting_pipeline(data_after_processing, test_size=0.2):
    """
    Run complete forecasting pipeline
    """
    # 1. Split data
    train_data, test_data, feature_cols = prepare_forecasting_data(data_after_processing, test_size=test_size)

    # 2. Train models
    results = train_models_with_cv(
        train_data=train_data,
        test_data=test_data,
        feature_cols=feature_cols,
        target_cols=['Quantity', 'Revenue']
    )
    return results

In [124]:
results = run_forecasting_pipeline(data_after_processing)


Using 47 features for modeling
Train period: 2021-01-01 00:00:00 to 2023-06-24 00:00:00
Test period: 2023-06-26 00:00:00 to 2023-12-31 00:00:00
Number of features: 47

Training models for Quantity
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5305
[LightGBM] [Info] Number of data points in the train set: 443, number of used features: 46
[LightGBM] [Info] Start training from score 0.167289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

16:13:41 - cmdstanpy - INFO - Chain [1] start processing
16:13:41 - cmdstanpy - INFO - Chain [1] done processing


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000715 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9123
[LightGBM] [Info] Number of data points in the train set: 882, number of used features: 47
[LightGBM] [Info] Start training from score 0.060230
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


16:13:42 - cmdstanpy - INFO - Chain [1] start processing
16:13:42 - cmdstanpy - INFO - Chain [1] done processing


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000611 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9137
[LightGBM] [Info] Number of data points in the train set: 1321, number of used features: 47
[LightGBM] [Info] Start training from score 0.039049
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


...

16:14:11 - cmdstanpy - INFO - Chain [1] start processing
16:14:11 - cmdstanpy - INFO - Chain [1] done processing


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000603 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9155
[LightGBM] [Info] Number of data points in the train set: 2199, number of used features: 47
[LightGBM] [Info] Start training from score -0.001805


16:14:16 - cmdstanpy - INFO - Chain [1] start processing
16:14:16 - cmdstanpy - INFO - Chain [1] done processing


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000872 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9159
[LightGBM] [Info] Number of data points in the train set: 2638, number of used features: 47
[LightGBM] [Info] Start training from score -0.000000


16:14:23 - cmdstanpy - INFO - Chain [1] start processing
16:14:23 - cmdstanpy - INFO - Chain [1] done processing


In [125]:
results

{'Quantity': {'models': {'lightgbm': <lightgbm.basic.Booster at 0x7fd7876b5ee0>,
   'prophet': <prophet.forecaster.Prophet at 0x7fd78643c1f0>,
   'sarimax': <statsmodels.tsa.statespace.sarimax.SARIMAXResultsWrapper at 0x7fd788631be0>},
  'forecasts': {'lightgbm': array([ 2.72028434,  1.04647695,  2.98230369,  4.93937692,  7.91618432,
           4.92803793,  4.92999384,  2.92264426,  7.75549822,  5.90584022,
           7.77655034,  2.03550193,  1.01249657,  2.92748312,  6.00985926,
           3.97173675,  1.01774917,  6.78025637,  6.79660669,  2.96465068,
           2.91652438,  3.96190712,  7.04040488,  3.96098365,  5.87117037,
           3.09110048,  1.02456643,  6.02448923,  1.06285192,  8.77358401,
           6.930866  ,  7.85781604,  4.09027344,  3.14227341,  7.87773693,
           3.97005475,  6.14743137,  8.88250654,  3.99720711,  5.96450957,
           7.93493288,  1.00359975,  4.07487768,  3.01095608,  4.90639013,
           7.93977594,  8.99794509,  4.803949  ,  1.15791238,  7